In [6]:
# Scraping hashtag on Twitter

import json
import codecs
import csv
import tweepy
from tweepy import OAuthHandler

class UTF8Recoder:

    def __init__(self, f, encoding):
        self.reader = codecs.getreader(encoding)(f)

    def __iter__(self):
        return self

    def next(self):
        return self.reader.next().encode('utf-8')

class UnicodeReader:

    def __init__(self, f, dialect=csv.excel, encoding='utf-8', **kwds):
        f = UTF8Recoder(f, encoding)
        self.reader = csv.reader(f, dialect=dialect, **kwds)

    def next(self):
        row = self.reader.next()
        return [unicode(s, 'utf-8') for s in row]

    def __iter__(self):
        return self

class UnicodeWriter:

    def __init__(self, f, dialect=csv.excel, encoding='utf-8', **kwds):
        self.queue = cStringIO.StringIO()
        self.writer = csv.writer(self.queue, dialect=dialect, **kwds)
        self.stream = f
        self.encoder = codecs.getincrementalencoder(encoding)()

    def writerow(self, row):
        self.writer.writerow([s.encode('utf-8') for s in row])
        data = self.queue.getvalue()
        data = data.decode('utf-8')
        data = self.encoder.encode(data)
        self.stream.write(data)
        self.queue.truncate(0)

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

# Carrega as credenciais da API do Twitter
# Nesse caso, as credenciais estão em um JSON salvo no mesmo diretório, que vai ser puxado pelo código
with open('credenciais_twitter.json') as cred_data:
    info = json.load(cred_data)
    consumer_key = info['consumer_key']
    consumer_secret = info['consumer_secret']
    access_token = info['access_token']
    access_token_secret = info['access_token_secret']

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)
max_tweets = 10
termo_de_busca = input("Qual o termo ou hashtag que você deseja buscar no twitter? ")
print()

with open('tweets_%s.csv' % termo_de_busca, 'w', encoding='utf-8') as f:
    wr = csv.writer(f, delimiter=';', lineterminator='\n')
    wr.writerow(["created_at", "tweet"])
    for tweet in tweepy.Cursor(api.search, q=termo_de_busca, rpp=100).items(max_tweets):
        print(tweet.created_at, tweet.text)
        wr.writerow([tweet.created_at, tweet.text])

Qual o termo ou hashtag que você deseja buscar no twitter? #bolsonaro

2019-02-14 16:47:29 RT @infoguerrachile: @chileconvoca Lo de #Bolsonaro los tiene con diarrea a los #ComuFachos amantes de #Maduro. Como ya no les llegan "ince…
2019-02-14 16:45:40 RT @Desgoverno_BR: @stanleyburburin Todos do clã #Bolsonaro. E o #SergioMoro
2019-02-14 16:45:25 RT @NovaRepublica1: #Bolsonaro reafirma em entrevista que seu compromisso é com a justiça: "Moro tem carta branca" https://t.co/nWLbRJkORD
2019-02-14 16:41:08 BASTARAM 45 DIAS PARA #BOLSONARO PROVAR QUE NÃO TEM CONDIÇÕES DE GOVERNAR O BRASIL
O capitão reformado Jair Messias… https://t.co/mPZc5sJVtG
2019-02-14 16:38:31 Aí já foi demais, né @ECantanhede? O erro até passava ok, pedir desculpinha em público com tweet puxa-saco e ganhar… https://t.co/VVPAzxFJD4
2019-02-14 16:35:21 @kirstenvdhul Die #VOC mentaliteit en #Rutte zijn manier van #normaal doen, normen en waarden🤔🤕   #nietnormaal… https://t.co/Shhosh2j80
2019-02-14 16:34:20 BASTARAM 45 DI